In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [5]:
path=r'D:\WORKS\PolimerSolve\Comsol\tables\1D'
input_filename = r'\out.csv'

In [12]:
out_name=input('Write file name: ')
if out_name!='':
    with open(path +r'\row'+ input_filename, 'r') as file:
        lines = file.readlines()
        
    cur_filename ='\\'+out_name+'.csv'
    with open(path+r'\cleared' + cur_filename, 'w') as file:
        i = 0
        for line in lines:
            if i > 3: file.write(line)
            i += 1

In [24]:
out_name=input('Write file name: ')
cur_filename ='\\'+out_name+'.csv'
df = pd.read_csv(path+r'\cleared' + cur_filename)
df = df.rename(columns={
    '% k1': 'k1',
    df.columns[1]: 'target',
})
df.head()

,k1,target
0,0.0,0.000000e+00
1,100.0,1.922426e-08
2,200.0,2.663444e-08
3,300.0,3.364097e-08
4,400.0,3.810817e-08


In [25]:
# scalling
df['target'] = 1 / df['target'].max() * df['target']

In [26]:
max_df = df[df['target'] >= 0.99]
fig = go.Figure()
fig.update_layout(margin={'r': 0, 'l': 0, 't': 0, 'b': 0})
fig.add_traces(
    go.Scatter(
        x=df['k1'],
        y=df['target'],
        hovertemplate='k1: %{x}<br>integral: %{y:.2f}',
        name='',
    ))